# 제7장 자주 사용하는 케라스 기능

**예제 코드7.1:드롭아웃 레이어 사용 예**

In [1]:
# 드롭아웃 레이어를 포함하는 Sequential 모델 예제
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Dropout


model = Sequential()
model.add(Dense(64, activation='relu', input_dim=20))
model.add(Dropout(0.5))  # Dropout을 rate=0.5로 적용
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

model.compile(
    loss='categorical_crossentropy',
    optimizer='SGD',
    metrics=['accuracy']
)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


**예제 코드7.2:BatchNormalization 레이어 사용 예**

In [2]:
# BatchNormalization 층을 포함하는 Sequential 모델 예제
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Activation, BatchNormalization


model = Sequential()
model.add(Dense(64, input_dim=20))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dense(64))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dense(10, activation='softmax'))

model.compile(
    loss='categorical_crossentropy',
    optimizer='SGD',
    metrics=['accuracy']
)

**예제 코드7.3:Lambda 레이어 사용 예**

In [3]:
# Lambda 레이어 사용 예제
from tensorflow.python.keras.layers import Input, Lambda
from tensorflow.python.keras.models import Model


model_in = Input(shape=(20,))
x = Lambda(lambda x: x/255.)(model_in)
x = Dense(64, activation='relu')(x)
model_out = Dense(10, activation='softmax')(x)

model = Model(inputs=model_in, outputs=model_out)
model.compile(
    loss='categorical_crossentropy',
    optimizer='SGD',
    metrics=['accuracy']
)

**예제 코드7.4:레이어의 인수를 사용해서 활성화 함수를 추가하는 예**

In [4]:
# Dense 레이어의 인수로 relu와 시그모이드를를 지정해서 활성화 함수를 추가
model = Sequential()
model.add(Dense(64, activation='relu', input_dim=20))
model.add(Dense(1, activation='sigmoid'))

model.compile(
    loss='binary_crossentropy',
    optimizer='SGD',
    metrics=['accuracy']
)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [5]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_8 (Dense)              (None, 64)                1344      
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 65        
Total params: 1,409
Trainable params: 1,409
Non-trainable params: 0
_________________________________________________________________


**예제 코드7.5:Activation 레이어를 호출해서 활성화 함수를 생성•추가하는 예**

In [6]:
# Activation 레이어를 호출해서 별도로 활성화 층을 추가하고 있다
from tensorflow.python.keras.layers import Activation
from tensorflow.python.keras.activations import relu


model = Sequential()
model.add(Dense(64, input_dim=20))
model.add(Activation('relu'))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(
    loss='binary_crossentropy',
    optimizer='SGD',
    metrics=['accuracy']
)

In [7]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_10 (Dense)             (None, 64)                1344      
_________________________________________________________________
activation_2 (Activation)    (None, 64)                0         
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 65        
_________________________________________________________________
activation_3 (Activation)    (None, 1)                 0         
Total params: 1,409
Trainable params: 1,409
Non-trainable params: 0
_________________________________________________________________


**예제 코드7.6:Clipped ReLU 의 예**

In [8]:
# Clipped ReLU의 예
from tensorflow.python.keras.layers import Activation
from tensorflow.python.keras.activations import relu


model = Sequential()
model.add(
    Dense(
        64, 
        input_dim=20, 
        activation=lambda x: relu(x, max_value=2)
    )
)
model.add(Dense(1, activation='sigmoid'))

model.compile(
    loss='binary_crossentropy',
    optimizer='SGD',
    metrics=['accuracy']
)

**예제 코드7.7：생성기 생성**

In [9]:
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator


# ImageDataGenerator 생성
# 대표적인 옵션을 지정한 예
gen = ImageDataGenerator(
    rescale=1/255.,  # 스케일 변환 
    rotation_range=90.,  # 데이터 확장 관련 
    width_shift_range=1.,
    height_shift_range=.5,
    shear_range=.8,
    zoom_range=.5,
    horizontal_flip=True,
    vertical_flip=True
)

**예제 코드7.8：iterator의 생과 데이터 취득**

In [10]:
# 폴더에서 이미지를 읽어들이고, iterator 생성
iters = gen.flow_from_directory(
    'img07',
    target_size=(32, 32),
    class_mode='binary',
    batch_size=5,
    shuffle=True
)

# iterator로부터 1 미니 배치분의 데이터를 취득
x_train_batch, y_train_batch = next(iters)

print('shape of x_train_batch:', x_train_batch.shape)
print('shape of y_train_batch:', y_train_batch.shape)

Found 25 images belonging to 2 classes.
shape of x_train_batch: (5, 32, 32, 3)
shape of y_train_batch: (5,)


**예제 코드7.9：iterator를 이용한 모델 학습**

In [11]:
import math
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Flatten, Dense, Conv2D


# 분류용 모델 구축
model = Sequential()
model.add(Conv2D(16, (3, 3), input_shape=(32, 32, 3)))
model.add(Flatten())
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(
    loss='binary_crossentropy',
    optimizer='rmsprop'
)


# 몇 미니 배치를 학습하면 1에폭인지 계산
steps_per_epoch = math.ceil(iters.samples/5)

# `fit_generator` 메서드에 iterator를 전달
histroy = model.fit_generator(
    iters, 
    steps_per_epoch=steps_per_epoch
)

5/5 [==============================] - 3s 544ms/step - loss: 1.1010
